# This notebook is used to reproduce the encountered errors

Initial remarks:
- lincs copy and lincs_SMILES copy are notebooks I've been experimenting with

## lincs.iypnb
Changes:
- imported Path from pathlib and changed the path for adata_in
- before, the directory was pointing to a directory in the installation of the package, also there was a folder called "project_folder", which didn't exist in the repository
- after applying those changes, the notebook runs without errors when the variable full is set to **False**
- code pasted below

In [ ]:
import numpy as np 
import pandas as pd
import scanpy as sc

from pathlib import Path
from tqdm.notebook import tqdm
sc.set_figure_params(dpi=100, frameon=False)
sc.logging.print_header()

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [ ]:
import os
os.chdir('./../')
from chemCPA.helper import rank_genes_groups_by_cov

In [ ]:
import warnings
warnings.filterwarnings('ignore')   

In [ ]:
full = False 
load_adata = True 

lincs_path = Path('/storage/kasperek/chemCPA/datasets/')
if full:
    adata_in = os.path.join(lincs_path, 'lincs_full.h5ad')
else:
    adata_in = os.path.join(lincs_path, 'lincs.h5ad')

adata = sc.read(adata_in) if load_adata else None

adata_out = ''.join(adata_in.split('.')[:-1]) + '_pp.h5ad'
adata_out

'/storage/kasperek/chemCPA/datasets/lincs_pp.h5ad'

In [ ]:
adata.obs['condition'] = adata.obs['pert_iname']
adata.obs['condition'] = adata.obs['condition'].str.replace('/','|')

adata.obs['cell_type'] = adata.obs['cell_id']
adata.obs['dose_val'] = adata.obs['pert_dose']
adata.obs['cov_drug_dose_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str) + '_' + adata.obs.dose_val.astype(str)
adata.obs['cov_drug_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str)
adata.obs['eval_category'] = adata.obs['cov_drug_name']
adata.obs['control'] = (adata.obs['condition'] == 'DMSO').astype(int)

# adata.obs['cov_drug_dose_name'] = adata.obs['cov_drug_dose_name'].str.replace('/','|')

In [ ]:
pd.crosstab(adata.obs.condition, adata.obs.cell_type)

cell_type,A375,A549,A673,AGS,ASC,ASC.C,BT20,CD34,CL34,CORL23,...,SW620,SW948,T3M10,THP1,TYKNU,U266,U937,VCAP,WSUDLCL2,YAPC
condition,,,,,,,,,,,,,,,,,,,,,
1B Parent,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2-methoxyestradiol,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
"3,6-dimethoxyflavone",5,3,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3-amino-benzamide,30,61,2,2,0,0,0,0,2,2,...,4,4,2,2,2,0,2,9,2,18
5-methoxy-alpha-methyltryptamine,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xanthoxyline,3,5,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,7,0,0
yohimbine,35,16,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,26,0,18
zacopride,9,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,0,0


In [ ]:
drug_abundance = adata.obs.condition.value_counts()
suff_drug_abundance = drug_abundance.index[drug_abundance>5]

In [ ]:
# Delete conditions isufficient # of observations
adata = adata[adata.obs.condition.isin(suff_drug_abundance)].copy()
adata 

AnnData object with n_obs × n_vars = 199620 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'batch', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'control', 'split', 'cov_drug_name', 'eval_category'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'rank_genes_groups_cov'

Calculate differential genes manually, such that the genes are the same per condition.

In [ ]:
%%time

de_genes = {}
de_genes_quick = {}

adata_df = adata.to_df()
adata_df['condition'] = adata.obs.condition
dmso = adata_df[adata_df.condition == "DMSO"].mean()

for cond, df in tqdm(adata_df.groupby('condition')): 
    if cond != 'DMSO':
        drug_mean = df.mean()
        de_50_idx = np.argsort(abs(drug_mean-dmso))[-50:]
        de_genes_quick[cond] = drug_mean.index[de_50_idx].values

if full: 
    de_genes = de_genes_quick

else:
    sc.tl.rank_genes_groups(
        adata,
        groupby='condition', 
        reference='DMSO',
        rankby_abs=True,
        n_genes=50
    )
    for cond in tqdm(np.unique(adata.obs['condition'])):
        if cond != 'DMSO':
            df = sc.get.rank_genes_groups_df(adata, group=cond)  # this takes a while
            de_genes[cond] = df['names'][:50].values


  0%|          | 0/1001 [00:00<?, ?it/s]

  0%|          | 0/1001 [00:00<?, ?it/s]

CPU times: user 1min 9s, sys: 1.73 s, total: 1min 11s
Wall time: 1min 11s


Mapping from `rank_genes_groups_cov` might cause problems when drug contains '_'

In [ ]:
def extract_drug(cond): 
    split = cond.split('_')
    if len(split) == 2: 
        return split[-1]
    return '_'.join(split[1:-1])

adata.obs['cov_drug_dose_name'].apply(lambda s: len(s.split('_'))).value_counts()
adata.obs['eval_category'].apply(lambda s: len(s.split('_'))).value_counts()

2    199620
Name: eval_category, dtype: int64

In [ ]:
adata.uns['rank_genes_groups_cov'] = {cat: de_genes_quick[extract_drug(cat)] for cat in adata.obs.eval_category.unique() if extract_drug(cat) != 'DMSO'}

In [ ]:
adata.uns['rank_genes_groups_cov']

{'A375_aminoglutethimide': array(['MAST2', 'CSNK2A2', 'ECH1', 'GATA2', 'SPAG7', 'KAT6A', 'CEBPA',
        'EPN2', 'SCAND1', 'SUPV3L1', 'UBQLN2', 'SKP1', 'ACOT9', 'RNH1',
        'USP22', 'MIF', 'PAF1', 'ACD', 'SRC', 'MLEC', 'PAK4', 'MRPS2',
        'TLR4', 'PCNA', 'CCNB1', 'CDC42', 'PLSCR3', 'PYCR1', 'ZNF586',
        'CDKN1B', 'TRAPPC6A', 'TEX10', 'HSPA8', 'SPP1', 'AKT1', 'INSIG1',
        'CIRBP', 'IKZF1', 'DPH2', 'KIAA0100', 'COG7', 'CCDC92', 'CLTB',
        'HOMER2', 'GRB7', 'MBTPS1', 'HSPA1A', 'ZNF274', 'BNIP3L', 'GAPDH'],
       dtype=object),
 'A375_nicardipine': array(['CCND3', 'PLK1', 'EAPP', 'SYNE2', 'ID2', 'IGF1R', 'DMTF1', 'BNIP3',
        'RAB31', 'SOX4', 'GRB7', 'NCOA3', 'PPIC', 'CCDC92', 'SOX2',
        'PSMB8', 'PEX11A', 'GNAI1', 'PSMD10', 'CDC25B', 'LRRC16A',
        'TSC22D3', 'HMGCR', 'SPP1', 'MAP4K4', 'CDKN1A', 'NGRN', 'UBQLN2',
        'SERPINE1', 'FOSL1', 'MAPKAPK2', 'RAB4A', 'MTFR1', 'SNAP25',
        'DNAJA3', 'PAF1', 'SPDEF', 'RALGDS', 'SIRT3', 'MIF', 'VPS28',


In [ ]:
adata.obs['split'] = 'train'

# take ood from top occurring perturbations to avoid losing data on low occ ones
ood_idx = sc.pp.subsample(
    adata[adata.obs.condition.isin(list(adata.obs.condition.value_counts().index[1:50]))],
    .1,
    copy=True
).obs.index
adata.obs['split'].loc[ood_idx] = 'ood'

# take test from a random subsampling of the rest
test_idx = sc.pp.subsample(
    adata[adata.obs.split != 'ood'],
    .16,
    copy=True
).obs.index
adata.obs['split'].loc[test_idx] = 'test'

In [ ]:
pd.crosstab(adata.obs['split'], adata.obs['condition'])

condition,1B Parent,2-methoxyestradiol,"3,6-dimethoxyflavone",3-amino-benzamide,5-methoxy-alpha-methyltryptamine,ABT-737,AG-490,AG-14361,AICA-ribonucleotide,ALW-II-38-3,...,veliparib,vinburnine,voglibose,wiskostatin,xanthohumol,xanthoxyline,yohimbine,zacopride,zaprinast,zileuton
split,,,,,,,,,,,,,,,,,,,,,
ood,0,0,0,0,0,64,0,0,0,0,...,135,0,0,0,0,0,0,0,0,0
test,2,17,4,59,4,125,23,66,50,89,...,188,24,20,7,9,9,60,26,33,51
train,20,100,30,343,14,583,130,272,249,509,...,936,150,105,58,58,53,266,81,175,279


In [ ]:
try: 
    del(adata.uns['rank_genes_groups'])  # too large
except: 
    print('All good.')

In [ ]:
# code compatibility
from scipy import sparse
adata.X = sparse.csr_matrix(adata.X)

In [ ]:
sc.write(adata_out, adata)

In [ ]:
print('all done.')

all done.


### Check that `adata.uns[rank_genes_groups_cov]` has all entries in `adata.obs.cov_drug_dose_name` as keys

In [ ]:
for i, k in enumerate(adata.obs.eval_category.unique()):
    try: 
        adata.uns['rank_genes_groups_cov'][k]
    except: 
        print(f"{i}: {k}") if 'DMSO' not in k else None

### Checking the same for the stored adata object

In [ ]:
adata_2 = sc.read(adata_out)

In [ ]:
for i, k in enumerate(adata_2.obs.eval_category.unique()):
    try: 
        adata_2.uns['rank_genes_groups_cov'][k]
    except: 
        print(f"{i}: {k}") if 'DMSO' not in k else None

In [ ]:
set(list(adata.uns['rank_genes_groups_cov'])) - set((list(adata_2.uns['rank_genes_groups_cov'])))

set()

In [ ]:
set((list(adata_2.uns['rank_genes_groups_cov']))) - set(list(adata.uns['rank_genes_groups_cov']))

set()

## When the parameter is set to full, the following error occurs:

In [ ]:
import numpy as np 
import pandas as pd
import scanpy as sc

from pathlib import Path
from tqdm.notebook import tqdm
sc.set_figure_params(dpi=100, frameon=False)
sc.logging.print_header()

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [ ]:
import os
os.chdir('./../')
from chemCPA.helper import rank_genes_groups_by_cov

In [ ]:
import warnings
warnings.filterwarnings('ignore')   

In [ ]:
full = True
load_adata = True 

lincs_path = Path('/storage/kasperek/chemCPA/datasets/')
if full:
    adata_in = os.path.join(lincs_path, 'lincs_full.h5ad')
else:
    adata_in = os.path.join(lincs_path, 'lincs.h5ad')

adata = sc.read(adata_in) if load_adata else None

adata_out = ''.join(adata_in.split('.')[:-1]) + '_pp.h5ad'
adata_out

'/storage/kasperek/chemCPA/datasets/lincs_full_pp.h5ad'

In [ ]:
adata.obs['condition'] = adata.obs['pert_iname']
adata.obs['condition'] = adata.obs['condition'].str.replace('/','|')

adata.obs['cell_type'] = adata.obs['cell_id']
adata.obs['dose_val'] = adata.obs['pert_dose']
adata.obs['cov_drug_dose_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str) + '_' + adata.obs.dose_val.astype(str)
adata.obs['cov_drug_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str)
adata.obs['eval_category'] = adata.obs['cov_drug_name']
adata.obs['control'] = (adata.obs['condition'] == 'DMSO').astype(int)

# adata.obs['cov_drug_dose_name'] = adata.obs['cov_drug_dose_name'].str.replace('/','|')

In [ ]:
pd.crosstab(adata.obs.condition, adata.obs.cell_type)

cell_type,A375,A549,A673,AGS,ASC,ASC.C,BT20,CD34,CL34,CORL23,...,SW620,SW948,T3M10,THP1,TYKNU,U266,U937,VCAP,WSUDLCL2,YAPC
condition,,,,,,,,,,,,,,,,,,,,,
(+)-3-(1-propyl-piperidin-3-yl)-phenol,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(+|-)-7-hydroxy-2-(N,N-di-n-propylamino)tetralin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"1,2,3,4,5,6-hexabromocyclohexane",5,5,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"1,2,3,4-tetrahydroisoquinoline",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"1,2-dichlorobenzene",3,6,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zonisamide,23,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,18
zopiclone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zosuquidar,20,6,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,10,0,17


In [ ]:
drug_abundance = adata.obs.condition.value_counts()
suff_drug_abundance = drug_abundance.index[drug_abundance>5]

In [ ]:
# Delete conditions isufficient # of observations
adata = adata[adata.obs.condition.isin(suff_drug_abundance)].copy()
adata 

AnnData object with n_obs × n_vars = 1023036 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'cov_drug_name', 'eval_category', 'control'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'cydata_pull'

Calculate differential genes manually, such that the genes are the same per condition.

In [ ]:
%%time

de_genes = {}
de_genes_quick = {}

adata_df = adata.to_df()
adata_df['condition'] = adata.obs.condition
dmso = adata_df[adata_df.condition == "DMSO"].mean()

for cond, df in tqdm(adata_df.groupby('condition')): 
    if cond != 'DMSO':
        drug_mean = df.mean()
        de_50_idx = np.argsort(abs(drug_mean-dmso))[-50:]
        de_genes_quick[cond] = drug_mean.index[de_50_idx].values

if full: 
    de_genes = de_genes_quick

else:
    sc.tl.rank_genes_groups(
        adata,
        groupby='condition', 
        reference='DMSO',
        rankby_abs=True,
        n_genes=50
    )
    for cond in tqdm(np.unique(adata.obs['condition'])):
        if cond != 'DMSO':
            df = sc.get.rank_genes_groups_df(adata, group=cond)  # this takes a while
            de_genes[cond] = df['names'][:50].values


  0%|          | 0/17990 [00:00<?, ?it/s]

CPU times: user 44.3 s, sys: 1.45 s, total: 45.8 s
Wall time: 45.4 s


Mapping from `rank_genes_groups_cov` might cause problems when drug contains '_'

In [ ]:
def extract_drug(cond): 
    split = cond.split('_')
    if len(split) == 2: 
        return split[-1]
    return '_'.join(split[1:-1])

adata.obs['cov_drug_dose_name'].apply(lambda s: len(s.split('_'))).value_counts()
adata.obs['eval_category'].apply(lambda s: len(s.split('_'))).value_counts()

2    1022382
3        654
Name: eval_category, dtype: int64

In [ ]:
adata.uns['rank_genes_groups_cov'] = {cat: de_genes_quick[extract_drug(cat)] for cat in adata.obs.eval_category.unique() if extract_drug(cat) != 'DMSO'}

KeyError: 'assinex'